In [39]:
import sys
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
import numpy as np

In [40]:
def MAPE(test, predict):
    mape = 0

    for i, j in zip(test['int_rate'].as_data_frame().values, predict.as_data_frame().values):
        mape += np.abs((i-j)/i)

    mape = (mape/test.shape[0])*100

    return mape


    

In [41]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 hour 48 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_akshatpant_yl16vs
H2O cluster total nodes:,1
H2O cluster free memory:,2.795 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [42]:
df_linear = h2o.import_file('../Data/Mice Data/data_after_mice.csv')
df_linear.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_length,annual_inc,issue_month,issue_year,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,application_type_JOINT,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source_Verified,verification_status_Verified,loan_status_Current,loan_status_Default,loan_status_Does_not_meet_the_credit_policy_Status_Charged_Off,loan_status_Does_not_meet_the_credit_policy_Status_Fully_Paid,loan_status_Fully_Paid,loan_status_In_Grace_Period,loan_status_Issued,loan_status_Late_16_30_days,loan_status_Late_31_120_days,pymnt_plan_y,initial_list_status_w,int_rate
0.119293,0.119293,0.118696,0.000858908,0.00388584,0.000238586,0.572605,0.000286303,0.0479796,0.000659689,0,2.39e-05,7.16e-05,0,0.325622,0.00199696,0.000214727,0,0,0.139837,0.139138,0.119293,0.0205439,0,0,0,0.00409461,0,0,0,0.651577,0.207569,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,10.65
0.0673226,0.0673226,0.0673226,0.00161574,0.00161116,0,0.807871,0.000323149,0.0541543,2.69e-05,0,0.000134645,8.08e-05,0,0.0454293,0.000253133,0.000107716,0,0,0.0271636,0.0271636,0.012292,0.0117187,0,0.00315285,2.99e-05,0.00322233,0,0,0,0.127401,0.557431,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,15.27
0.152472,0.152472,0.152472,0.00228709,0.0053575,0.000635302,0.778372,0.000762362,0.127759,0.000553983,0,0.00012706,0.00012706,0,0.187795,0.00625772,0.000635302,0,0,0.190823,0.190822,0.152472,0.03835,0,0,0,0.0412889,0,0,0,0.415297,0.0317651,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,15.96
0.155753,0.155753,0.155753,0.000560711,0.00528485,0.000155753,0.766305,0.000186904,0.0313219,0.000311506,0,1.56e-05,0.000155753,0,0.0871905,0.000327081,0.000576286,0,0,0.190428,0.190428,0.155753,0.034411,0.000264313,0,0,0.00556786,0,0,0,0.42371,0.232072,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,13.49
0.032231,0.032231,0.032231,0.000644621,0.000728314,1.07e-05,0.859494,0.000128924,0.0216055,0.000192742,0,0,0.000161155,0,0.298492,0.000579084,0.00040826,0.00823933,0.00823933,0.0348328,0.0348328,0.0239917,0.0108411,0,0,0,0.000728314,0,0,0,0.125454,0.386772,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,12.69
0.0993489,0.0993489,0.0993489,0.000715312,0.00310882,5.96e-05,0.715312,0.000238437,0.0399581,0.000222541,0,5.96e-05,0.000178828,0,0.158223,0.000562315,0.000238437,0,0,0.111894,0.111894,0.0993489,0.0125454,0,0,0,0.00319963,0,0,0.0853804,0.0885794,0.61795,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,7.9
0.106246,0.106246,0.106246,0.000910684,0.00258148,0.000121424,0.71343,0.000182137,0.0305231,0.000356836,0,1.52e-05,0.000106246,0,0.269046,0.00129924,0.000166959,0.0286736,0.0286736,0.123501,0.123501,0.0775728,0.0459287,0,0,0,0.00258148,0,0,0.00101693,0.545059,0.215528,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0

In [43]:
df = h2o.import_file('/Users/akshatpant/Desktop/Rishika/data_after_mice.csv')
df

Parse progress: |█████████████████████████████████████████████████████████| 100%


loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,issue_month,issue_year,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source_Verified,verification_status_Verified,loan_status_Current,loan_status_Default,loan_status_Does_not_meet_the_credit_policy_Status_Charged_Off,loan_status_Does_not_meet_the_credit_policy_Status_Fully_Paid,loan_status_Fully_Paid,loan_status_In_Grace_Period,loan_status_Issued,loan_status_Late_16_30_days,loan_status_Late_31_120_days,pymnt_plan_y,initial_list_status_w,application_type_JOINT
5000,5000,4975,36,10.65,162.87,10,24000,12,2011,27.65,0,1,3,0,13648,83.7,9,0,0,5861.07,5831.78,5000,861.07,0,0,0,171.62,0,0,0,27310,8700,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0
2500,2500,2500,60,15.27,59.83,0,30000,12,2011,1,0,5,3,0,1687,9.4,4,0,0,1008.71,1008.71,456.46,435.17,0,117.08,1.11,119.66,0,0,0,4731,20700,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2400,2400,2400,36,15.96,84.33,10,12252,12,2011,8.72,0,2,2,0,2956,98.5,10,0,0,3003.65,3003.65,2400,603.65,0,0,0,649.91,0,0,0,6537,500,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
10000,10000,10000,36,13.49,339.31,10,49200,12,2011,20,0,1,10,0,5598,21,37,0,0,12226.3,12226.3,10000,2209.33,16.97,0,0,357.48,0,0,0,27204,14900,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
3000,3000,3000,60,12.69,67.79,1,80000,12,2011,17.94,0,0,15,0,27783,53.9,38,766.9,766.9,3242.17,3242.17,2233.1,1009.07,0,0,0,67.79,0,0,0,11677,36000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
5000,5000,5000,36,7.9,156.46,3,36000,12,2011,11.2,0,3,9,0,7963,28.3,12,0,0,5631.38,5631.38,5000,631.38,0,0,0,161.03,0,0,4297,4458,31100,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
7000,7000,7000,60,15.96,170.08,8,47004,12,2011,23.51,0,1,7,0,17726,85.6,11,1889.15,1889.15,8136.84,8136.84,5110.85,3025.99,0,0,0,170.08,0,0,67,35911,14200,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3000,3000,3000,36,18.64,109.43,9,48000,12,2011,5.35,0,2,4,0,8221,87.5,4,0,0,3938.14,3938.14,3000,938.14,0,0,0,111.34,0,0,0,27957,5200,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
5600,5600,5600,60,21.28,152.39,4,40000,12,2011,5.55,0,2,11,0,5210,32.6,13,0,0,646.02,646.02,162.02,294.94,0,189.06,2.09,152.39,0,0,82,26755,7900,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5375,5375,5350,60,12.69,121.45,0,15000,12,2011,18.08,0,0,2,0,9279,36.5,3,0,0,1476.19,1469.34,673.48,533.42,0,269.29,2.52,121.45,0,0,0,10619,24200,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [44]:
df.shape

(887379, 92)

In [45]:
train, valid, test = df.split_frame(
    ratios=[0.6,0.2], 
    seed=1234, 
    destination_frames=['train.hex','valid.hex','test.hex']
)

In [46]:
train_linear, valid_linear, test_linear = df_linear.split_frame(
    ratios=[0.6,0.2], 
    seed=1234, 
    destination_frames=['train.hex','valid.hex','test.hex']
)

In [47]:
df.columns

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'emp_length',
 'annual_inc',
 'issue_month',
 'issue_year',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'sub_grade_A2',
 'sub_grade_A3',
 'sub_grade_A4',
 'sub_grade_A5',
 'sub_grade_B1',
 'sub_grade_B2',
 'sub_grade_B3',
 'sub_grade_B4',
 'sub_grade_B5',
 'sub_grade_C1',
 'sub_grade_C2',
 'sub_grade_C3',
 'sub_grade_C4',
 'sub_grade_C5',
 'sub_grade_D1',
 'sub_grade_D2',
 'sub_grade_D3',
 'sub_grade_D4',
 'sub_grade_D5',
 'sub_grade_E1',
 'sub_grade_E2',
 'sub_grade_E3'

In [48]:
# predictors = [i for i in df.columns if 'grade' in i]
# predictors.append('loan_status_Does_not_meet_the_credit_policy_Status_Charged_Off')
# predictors.append('loan_status_Does_not_meet_the_credit_policy_Status_Fully_Paid')
# predictors.append('inq_last_6mths')
# predictors

predictors = ['grade_C','grade_D','grade_E',
        'grade_F','grade_G','total_rec_int',
        'total_pymnt_inv','funded_amnt_inv','sub_grade_B5',
        'sub_grade_C5','sub_grade_C4','sub_grade_C3','sub_grade_B4','sub_grade_D5']

predictors

['grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'total_rec_int',
 'total_pymnt_inv',
 'funded_amnt_inv',
 'sub_grade_B5',
 'sub_grade_C5',
 'sub_grade_C4',
 'sub_grade_C3',
 'sub_grade_B4',
 'sub_grade_D5']

In [49]:
response = 'int_rate'

### Random Forest

In [50]:
rf = H2ORandomForestEstimator(ntrees = 20, max_depth = 60, stopping_rounds = 2, 
                             score_each_iteration = True, seed = 1000000)
rf.train(x = predictors, y = response, training_frame = train, validation_frame = valid)
rf

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1553282803100_25


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.7280196567362235
RMSE: 0.8532406792554041
MAE: 0.4694499951492592
RMSLE: 0.07120868905774304
Mean Residual Deviance: 0.7280196567362235

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 0.6750968130417894
RMSE: 0.8216427526862203
MAE: 0.4529064636679627
RMSLE: 0.06868108253567803
Mean Residual Deviance: 0.6750968130417894
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2019-03-22 17:16:45,0.000 sec,0.0,nan,nan,nan,nan,nan,nan
,2019-03-22 17:16:48,2.546 sec,1.0,1.0860400,0.5470559,1.1794828,1.0843989,0.5459920,1.1759209
,2019-03-22 17:16:51,5.489 sec,2.0,1.0646322,0.5646972,1.1334417,0.9539208,0.5249704,0.9099648
,2019-03-22 17:16:54,8.653 sec,3.0,1.0338211,0.5448451,1.0687861,0.9047159,0.4966632,0.8185108
,2019-03-22 17:16:57,11.534 sec,4.0,1.0084388,0.5312170,1.0169488,0.8803583,0.4817565,0.7750308
---,---,---,---,---,---,---,---,---,---
,2019-03-22 17:17:31,45.242 sec,16.0,0.8669432,0.4755830,0.7515905,0.8252575,0.4546844,0.6810499
,2019-03-22 17:17:34,48.196 sec,17.0,0.8624415,0.4727275,0.7438053,0.8236998,0.4529539,0.6784814
,2019-03-22 17:17:37,51.208 sec,18.0,0.8585743,0.4702756,0.7371498,0.8225178,0.4516540,0.6765355
,2019-03-22 17:17:39,54.020 sec,19.0,0.8567470,0.4718125,0.7340155,0.8228887,0.4542873,0.6771458



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
grade_E,35637936.0000000,1.0,0.2545163
grade_D,24976936.0000000,0.7008525,0.1783784
grade_F,23621052.0000000,0.6628064,0.1686951
total_rec_int,14866897.0000000,0.4171649,0.1061753
grade_C,14356702.0000000,0.4028489,0.1025316
grade_G,8253041.5,0.2315802,0.0589410
total_pymnt_inv,4400846.0,0.1234877,0.0314296
funded_amnt_inv,3356496.0,0.0941832,0.0239712
sub_grade_B5,3133298.2500000,0.0879203,0.0223772
sub_grade_B4,2782927.0,0.0780889,0.0198749


In [51]:
test_predict = rf.predict(test)
test_predict

drf prediction progress: |████████████████████████████████████████████████| 100%


predict
13.4925
12.388
9.9725
8.248
11.764
8.8615
14.6415
12.44
12.851
11.8435


In [52]:
mape = 0

for i, j in zip(test['int_rate'].as_data_frame().values, test_predict.as_data_frame().values):
    mape += np.abs((i-j)/i)
    
mape = (mape/test.shape[0])*100

mape    

array([3.80598074])

### Linear Regression

In [53]:
glm = H2OGeneralizedLinearEstimator(family = 'gaussian', lambda_search = True)
glm.train(x = predictors, y = response, training_frame = train_linear, validation_frame = valid_linear)
glm

glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1553282803100_26


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 1.5241704166319763
RMSE: 1.2345729693428316
MAE: 0.9180440273411529
RMSLE: 0.11064334458291161
R^2: 0.9205645260251174
Mean Residual Deviance: 1.5241704166319763
Null degrees of freedom: 532514
Residual degrees of freedom: 532500
Null deviance: 10217646.711212408
Residual deviance: 811643.6094127769
AIC: 1735672.6981188566

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 1.5248883168294192
RMSE: 1.2348636835009033
MAE: 0.9190480265376683
RMSLE: 0.110585363561804
R^2: 0.920408655455999
Mean Residual Deviance: 1.5248883168294192
Null degrees of freedom: 177376
Residual degrees of freedom: 177362
Null deviance: 3398379.339718937
Residual deviance: 270480.1149742519
AIC: 578245.2322250835
Scoring His

,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test
,2019-03-22 17:19:31,0.000 sec,1,.41E1,1,19.1875285,19.1590756
,2019-03-22 17:19:31,0.073 sec,2,.37E1,3,18.8083694,18.7819807
,2019-03-22 17:19:31,0.104 sec,3,.34E1,4,18.3423965,18.3177927
,2019-03-22 17:19:31,0.139 sec,4,.31E1,5,17.5352392,17.5113782
,2019-03-22 17:19:31,0.170 sec,5,.28E1,5,16.7302194,16.7071161
---,---,---,---,---,---,---,---
,2019-03-22 17:19:34,3.169 sec,75,.41E-2,15,1.5245547,1.5252463
,2019-03-22 17:19:34,3.206 sec,76,.38E-2,15,1.5244304,1.5251295
,2019-03-22 17:19:34,3.241 sec,77,.34E-2,15,1.5243280,1.5250338
,2019-03-22 17:19:34,3.281 sec,78,.31E-2,15,1.5242421,1.5249542



See the whole table with table.as_data_frame()


In [54]:
test_predict = glm.predict(test)
test_predict

glm prediction progress: |████████████████████████████████████████████████| 100%


predict
14.2323
12.0853
8.73586
8.608
9.05307
8.74589
16.0438
12.0222
12.8154
10.0516


In [55]:
MAPE(test, test_predict)

array([8.53166087])

### Neural Network

In [57]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

In [58]:
results = []

for ep in epochs:
    for batch in batch_size:
        print('epoch: {}, batch: {}'.format(ep, batch))
        dl = H2ODeepLearningEstimator(hidden = [50, 25], activation = 'Tanh', 
                              distribution = 'gaussian', mini_batch_size = batch, epochs = ep)
        dl.train(x = predictors, y = response, training_frame = train, validation_frame = valid)
        
        test_predict = dl.predict(test)
        
        mape = MAPE(test, test_predict)
        
        results.append('epoch: {}, batch: {}, MAPE: {}'.format(ep, batch, mape))
        

epoch: 10, batch: 10
deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
epoch: 10, batch: 20
deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
epoch: 10, batch: 40
deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
epoch: 10, batch: 60
deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
epoch: 10, batch: 80
deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
epoch: 10, batch: 100
deeplearning Model Build progress: |███████████████████████████

In [59]:
results

['epoch: 10, batch: 10, MAPE: [5.8435322]',
 'epoch: 10, batch: 20, MAPE: [6.65448459]',
 'epoch: 10, batch: 40, MAPE: [5.56768898]',
 'epoch: 10, batch: 60, MAPE: [5.4776815]',
 'epoch: 10, batch: 80, MAPE: [6.23096064]',
 'epoch: 10, batch: 100, MAPE: [5.8803101]',
 'epoch: 50, batch: 10, MAPE: [5.55235419]',
 'epoch: 50, batch: 20, MAPE: [5.68400437]',
 'epoch: 50, batch: 40, MAPE: [5.66936924]',
 'epoch: 50, batch: 60, MAPE: [6.05826013]',
 'epoch: 50, batch: 80, MAPE: [5.67016735]',
 'epoch: 50, batch: 100, MAPE: [5.57100822]',
 'epoch: 100, batch: 10, MAPE: [5.79249894]',
 'epoch: 100, batch: 20, MAPE: [6.01402153]',
 'epoch: 100, batch: 40, MAPE: [5.67332163]',
 'epoch: 100, batch: 60, MAPE: [6.09957444]',
 'epoch: 100, batch: 80, MAPE: [5.56890193]',
 'epoch: 100, batch: 100, MAPE: [6.01571599]']